Read from music dataset and write it into 

In [2]:
!pip install pandas
SEPERATOR = "|"

In [ ]:
# make working directory if not present
!mkdir -p raw/datasets/
%cd ./raw/datasets/
# fetch datasource for 
!wget -N https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/CDs_and_Vinyl.jsonl.gz
!wget -N https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/meta_categories/meta_CDs_and_Vinyl.jsonl.gz
# making working directory for music
!mkdir -p music

In [3]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

# add filter during loading
def getDF(path, limit=-1, filter=None):
  i = 0
  df = {}
  for d in parse(path):
    if(limit >0):
      if(i>=limit):
        break
    if(filter and filter(d)):
      #print(d["title"] + "|"+str(d["average_rating"])+"|"+str(d["rating_number"]))
      continue
    else:
      df[i] = d
      i += 1

  return pd.DataFrame.from_dict(df, orient='index')

In [4]:
def filterByRating(data, average_rating=3, rating_number=20):
    if(data['average_rating'] and  data['rating_number']):
        return data['average_rating']< average_rating or data['rating_number'] < rating_number
    else:
        return False

def filterByASIN(asin_list=[]):
    def filterInList(data):
        if(data['asin']):
            return (data['asin'] not in asin_list)
        else:
            return True
    return filterInList

def filterByParentASIN(asin_list=[]):
    def filterInList(data):
        if(data['parent_asin']):
            return (data['parent_asin'] not in asin_list)
        else:
            return True
    return filterInList

In [5]:
# # Step 1: Read the json file into a DataFrame
# # input_file = 'meta_CDs_and_Vinyl2018.json.gz'
# input_file = 'meta_CDs_and_Vinyl.jsonl.gz'
# df = getDF(input_file, 1000, filterByRating) #enabled for 2023 data

# # set music_id according to the since the model requires sequence for music_id
# # df["music_id"] = df["parent_asin"].apply(hashToInt)
# df.sort_values(by="rating_number")
# df["music_id"] = df.reindex().index +1

# #print(df.head())
# # Filter rows where 'category' is non-empty or an empty array
# # filtered_df = df[df['category'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
# filtered_content = df[df['categories'].apply(lambda x: isinstance(x, list) and len(x) > 0)]#enabled for 2023 data
# # Show the filtered DataFrame
# print(filtered_content['categories'].count())

# # convert id into int

# # Convert the list in 'category' to a string with '|' as a separator
# # filtered_df['genres'] = filtered_df['category'].apply(lambda x: '|'.join(map(str, x)))
# filtered_content['genres'] = filtered_content['categories'].apply(lambda x: '|'.join(map(str, x)))#enabled for 2023 data
# # Step 2: Pick necessary columns
# # selected_content = ["asin", "title", "genres"]
# selected_content = ["music_id", "title", "genres"]#enabled for 2023 data
# df_content = filtered_content[selected_content]


# # Step 3: Save the transformed DataFrame to a new CSV file with a custom separator and no header
# # content_file = 'music/music2018.dat'
# content_file = 'music/music2023.dat'

# temp_content_file = content_file+'_temp'
# df_content.to_csv(content_file, sep=SEPERATOR, index=False, header=False)
# # with open(temp_content_file, 'r') as file:
# #     file_data = file.read()
# # file_data = file_data.replace(',', '::')
# # with open(content_file, 'w') as file:
# #     file.write(file_data) 

In [7]:
# # Step 1: Read the json file into a DataFrame
# # input_file = 'CDs_and_Vinyl2018.json.gz'
# input_file = 'CDs_and_Vinyl.jsonl.gz'

# # asin_list=filtered_df["asin"].to_list()
# asin_list=filtered_content["parent_asin"].to_list()#enabled for 2023 data
# rating_sum = filtered_content["rating_number"].sum()
# print(f"Fetch comments for  {len(asin_list)} products and supposed ratings to be recieved= {rating_sum}")
# # df = getDF(input_file,1000, filterByASIN(asin_list))
# df = getDF(input_file,rating_sum, filterByParentASIN(asin_list))#enabled for 2023 data
# # df["music_id"] = df["parent_asin"].apply(hashToInt)
# df = df.set_index('parent_asin').join(filtered_content.set_index('parent_asin'), rsuffix="meta_")

# # TODO: sum the ratings by parent_asin to see the different in rating_number and actually ratings counted

# # Step 2: Pick necessary columns
# # selected_rating = ["asin", "user_id","rating","timestamp"]
# selected_rating = ["user_id", "music_id", "rating","timestamp"] #enabled for 2023 data
# df_rating = df[selected_rating]

# #step 3: Calculate average rating

# # rating_counts = df_rating['rating'].value_counts().reset_index()
# # rating_counts.columns = ['rating', 'count']

# # total_sum_of_ratings = df_rating['rating'].sum()

# # rating_counts['avg_rating'] = rating_counts['count'] / total_sum_of_ratings

# # df_rating = df_rating.merge(rating_counts[['rating', 'avg_rating']], on='rating', how='left')

# avg_ratings = df_rating.groupby('music_id')['rating'].mean().reset_index()
# avg_ratings.columns = ['music_id', 'avg_rating']

# # # Merge the average rating back into the original df_rating
# df_rating = df_rating.merge(avg_ratings, on='music_id', how='left')
# # avg_ratings.to_csv('temp_avg_ratings.csv', index=False)

# # Step 4: Save the transformed DataFrame to a new CSV file with a custom separator and no header
# # rating_file = 'music/rating2018.dat'
# rating_file = 'music/rating2023.dat'
# temp_rating_file = rating_file+'_temp'
# df_rating.to_csv(rating_file, sep=SEPERATOR, index=False, header=False)
# # with open(temp_rating_file, 'r') as file:
# #     file_data = file.read()
# # file_data = file_data.replace(',', '::')
# # with open(rating_file, 'w') as file:
# #     file.write(file_data)


import pandas as pd
import gzip
import json
import hashlib

SEPARATOR = '::'

def getDF(filename, nrows=None, filter_func=None):
    data = []
    with gzip.open(filename, 'r') as f:
        for i, line in enumerate(f):
            if nrows is not None and i >= nrows:
                break
            item = json.loads(line.strip())
            if filter_func is None or filter_func(item):
                data.append(item)
    return pd.DataFrame(data)

def filterByRating(item):
    return 'rating_number' in item and item['rating_number'] > 0

def filterByParentASIN(asin_list):
    return lambda item: item.get('parent_asin') in asin_list

def hashToInt(s):
    return int(hashlib.md5(s.encode('utf-8')).hexdigest(), 16) % (10 ** 8)

# Step 1: Process CDs_and_Vinyl2018.json.gz to calculate average ratings
input_file_ratings = 'CDs_and_Vinyl2018.json.gz'
df_ratings = getDF(input_file_ratings, filterByRating)
df_ratings["music_id"] = df_ratings["asin"].apply(hashToInt)

avg_ratings = df_ratings.groupby('music_id')['overall'].mean().reset_index()
avg_ratings.columns = ['music_id', 'avg_rating']

# Step 2: Process meta_CDs_and_Vinyl.jsonl.gz
input_file_meta = 'meta_CDs_and_Vinyl.jsonl.gz'
df_meta = getDF(input_file_meta, 1000, filterByRating)

df_meta.sort_values(by="rating_number")
df_meta["music_id"] = df_meta.reindex().index + 1

filtered_content = df_meta[df_meta['categories'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
filtered_content['genres'] = filtered_content['categories'].apply(lambda x: '|'.join(map(str, x)))

selected_content = ["music_id", "title", "genres"]
df_content = filtered_content[selected_content]

# Step 3: Save the transformed DataFrame to a new CSV file with a custom separator and no header
content_file = 'music/music2023.dat'
df_content.to_csv(content_file, sep=SEPARATOR, index=False, header=False)

# Step 4: Process CDs_and_Vinyl.jsonl.gz to fetch comments and ratings
input_file_comments = 'CDs_and_Vinyl.jsonl.gz'
asin_list = filtered_content["parent_asin"].to_list()
rating_sum = filtered_content["rating_number"].sum()
print(f"Fetch comments for {len(asin_list)} products and supposed ratings to be received = {rating_sum}")
df_comments = getDF(input_file_comments, rating_sum, filterByParentASIN(asin_list))

df_comments = df_comments.set_index('parent_asin').join(filtered_content.set_index('parent_asin'), rsuffix="meta_")

selected_rating = ["user_id", "music_id", "rating", "timestamp"]
df_rating = df_comments[selected_rating]

# Step 5: Calculate average rating and merge into df_rating
avg_ratings = df_rating.groupby('music_id')['rating'].mean().reset_index()
avg_ratings.columns = ['music_id', 'avg_rating']

df_rating = df_rating.merge(avg_ratings, on='music_id', how='left')

# Step 6: Save the transformed DataFrame to a new CSV file with a custom separator and no header
rating_file = 'music/rating2023.dat'
df_rating.to_csv(rating_file, sep=SEPARATOR, index=False, header=False)

# Step 7: Merge average ratings into df_content and save to music2023.dat
df_content = df_content.merge(avg_ratings, on='music_id', how='left')
df_content.to_csv(content_file, sep=SEPARATOR, index=False, header=False)

print(f"Processed {len(df_content)} items and saved to {content_file}")

FileNotFoundError: [Errno 2] No such file or directory: 'CDs_and_Vinyl2018.json.gz'

In [9]:
import os
RATINGS_FILE_NAME = "rating2023.dat"
MOVIES_FILE_NAME = "music2023.dat"
RATINGS_DATA_COLUMNS = ["UserID", "MovieID", "Rating", "Timestamp"]
MOVIES_DATA_COLUMNS = ["MovieID", "Title", "Genres"]
OUTPUT_TRAINING_DATA_FILENAME = "train_music_amz2023.tfrecord"
OUTPUT_TESTING_DATA_FILENAME = "test_music_amz2023.tfrecord"

print("First few lines for ratings and movies:")

# read the data 
ratings_df = pd.read_csv(
    os.path.join(os.curdir, "music",RATINGS_FILE_NAME),
    sep=SEPERATOR,
    names=RATINGS_DATA_COLUMNS,
    encoding="unicode_escape")

print(ratings_df.head())

movies_df = pd.read_csv(
    os.path.join(os.curdir, "music", MOVIES_FILE_NAME),
    sep=SEPERATOR,
    names=MOVIES_DATA_COLUMNS,
    encoding="unicode_escape")  # May contain unicode. Need to escape.

print(movies_df.head())

First few lines for ratings and movies:
                              UserID  MovieID         Rating  Timestamp
AEVQ3KP55X4XECXWMHN6DHIDBYFQ     179      5.0  1406805139000   4.312500
AFAIJYOUO3NAWLBDIKTQSC3DASWA     541      5.0  1149955013000   4.827586
AHB5CGLYN3Y6NIPHNQLYFJT2W2PQ     981      5.0  1524455984589   4.591837
AE7BV6IMNPZ3F266H7PXMH3BZQNQ     266      5.0  1637093229001   4.260870
AGEFGQNUBPAGS4JYBD664PSZKDNQ     728      5.0  1351747347000   4.819672
   MovieID                     Title  \
0        1      Release Some Tension   
1        2         Somewhere in Time   
2        3  Heavy Hearts, Hard Fists   
3        4      Malice in Wonderland   
4        5                Elvis Live   

                                          Genres  
0           CDs & Vinyl|Dance & Electronic|House  
1           CDs & Vinyl|Soundtracks|Movie Scores  
2                                CDs & Vinyl|Pop  
3  CDs & Vinyl|Rock|Progressive|Progressive Rock  
4                         CDs & 